In [1]:
import sys
sys.path.append("../src")
import gym
import pybullet_envs
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import wandb
from config import *
from replay_buffer import *
from networks import *
from agent import *
import pgdrive

config = dict(
  learning_rate_actor = ACTOR_LR,
  learning_rate_critic = ACTOR_LR,
  batch_size = BATCH_SIZE,
  architecture = "SAC",
  infra = "Colab",
  env = ENV_NAME
)

wandb.init(
  project=f"tensorflow2_sac_{ENV_NAME.lower()}",
  tags=["SAC", "FCL", "RL"],
  config=config,
)

env = gym.make(ENV_NAME)
agent = Agent(env)

scores = []
evaluation = True

if PATH_LOAD is not None:
    print("loading weights")
    observation = env.reset()
    action, log_probs = agent.actor.get_action_log_probs(observation[None, :], False)
    agent.actor(observation[None, :])
    agent.critic_0(observation[None, :], action)
    agent.critic_1(observation[None, :], action)
    agent.critic_value(observation[None, :])
    agent.critic_target_value(observation[None, :])
    agent.load()
    print(agent.replay_buffer.buffer_counter)
    print(agent.replay_buffer.n_games)

Successfully registered the following environments: ['PGDrive-test-v0', 'PGDrive-validation-v0', 'PGDrive-v0', 'PGDrive-10envs-v0', 'PGDrive-1000envs-v0', 'PGDrive-training0-v0', 'PGDrive-training1-v0', 'PGDrive-training2-v0'].


wandb: Currently logged in as: anton-ai (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [2]:
for _ in tqdm(range(MAX_GAMES)):
    start_time = time.time()
    states = env.reset()
    done = False
    score = 0
    while not done:
        action = agent.get_action(states)
        new_states, reward, done, info = env.step(action)
        score += reward
        agent.add_to_replay_buffer(states, action, reward, new_states, done)
        agent.learn()
        states = new_states
    
    scores.append(score)
    agent.replay_buffer.update_n_games()

    wandb.log({'Game number': agent.replay_buffer.n_games, '# Episodes': agent.replay_buffer.buffer_counter, 
               "Average reward": round(np.mean(scores[-10:]), 2), \
                      "Time taken": round(time.time() - start_time, 2)})
    
    if (_ + 1) % SAVE_FREQUENCY == 0:
        print("saving...")
        agent.save()
        print("saved")

  0%|          | 6/100000 [17:42<4917:14:11, 177.03s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-8f485f7f2afa>", line 11, in <module>
    agent.learn()
  File "../src/agent.py", line 108, in learn
    new_policy_actions, log_probs = self.actor.get_action_log_probs(states, reparameterization_trick=True)
  File "../src/networks.py", line 88, in get_action_log_probs
    log_probs = normal_distr.log_prob(actions) - tf.math.log(1 - tf.math.pow(action,2) + self.epsilon)
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow_probability/python/distributions/distribution.py", line 866, in log_prob
    return self._call_log_prob(value, name, **kwargs)
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow_probability/python/distributions/distribution.py", line

TypeError: object of type 'NoneType' has no len()